[Natarajan Krishnaswami](mailto:natarajan@krishnaswami.org)  
W261 Machine Learning at Scale  
Spring 2016 / Section 2  
Week 1 Async Quiz  
2016-01-01

# Map

In [80]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")

findword = sys.argv[1]
# loop over lines in file iterable
for line in sys.stdin:
    # examine words
    for word in WORD_RE.findall(line):
        # to find matches to the query word
        if word == findword:
            count += 1
            # counting lines rather than occurences, so skip to next line after a match
            break
print count

Overwriting mapper.py


In [78]:
!chmod a+x mapper.py

# Reduce
Since we don't pass the chunk filename to the reducer, we can't give good error diagnostics.

In [34]:
%%writefile reducer.py
#!/usr/bin/python
import sys, os
sum = 0
for line in sys.stdin:
    try:
        sum += int(line.strip())
    except:
        print >>sys.stderr, "Error parsing {0} as int".format(line)
print sum

Overwriting reducer.py


In [35]:
!chmod a+x reducer.py

# Write script to file
I shuffled the script lines a bit to plug in the usage message.  
I tried as an experiment avoiding a temporary copy on disk and limiting parallelism to the number of CPUs by by switching to GNU `parallel`. However, using `parallel` misses the point of the activity so I changed it back.

In [90]:
%%writefile pGrepCount.sh
usage()
{
    echo Parallel grep
    echo "usage: pGrepCount [filename] [word] [chunksize]"
    echo "Searches file <filename> and counts the number of lines containing <word>."
    echo "Note: <filename> will be split in chunks of up to <chunksize> bytes each."
    echo "Lines containing <word> will be counted for each chunk in parallel"
}

if [ $# -lt 3 ]; then
    usage >&2
    exit 1
fi

ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted

#CLEANUP
rm -f ./${CHUNK_FILE_PREFIX}*
    
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -C $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD <$file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
# Preceding is approx equiv to 
#   numOfInstances=$(parallel --pipe --line-buffer --block-size $BLOCK_SIZE --no-notice ./mapper.py $FIND_WORD < $ORIGINAL_FILE | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


# Run the file

In [85]:
!chmod a+x pGrepCount.sh
!./pGrepCount.sh

Parallel grep
usage: pGrepCount [filename] [word] [chunksize]
Searches file <filename> and counts the number of lines containing <word>.
Note: <filename> will be split in chunks of up to <chunksize> bytes each.
Lines containing <word> will be counted for each chunk in parallel


In [91]:
!./pGrepCount.sh LICENSE.txt COPYRIGHT 4096

found [11] [COPYRIGHT] in the file [LICENSE.txt]


## Sanity check

In [71]:
!grep COPYRIGHT License.txt | wc -l

11
